<a href="https://colab.research.google.com/github/danielbauer1979/FI830/blob/main/FI830_HW5_MC_Simulations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import statistics
import scipy.stats as st

# Q2. Monte Carlo Option Pricing (I): European Put Option
Consider the European Put Option with $S_0=K=3977.19,\ \sigma=12\%,\ r=5\%,$ and expiration $T=1.$

In [ ]:
def MC_BSPut(S0,K,T,sigma,r,N):
  Z = np.random.normal(0,1,N)
  S = payoff = [0]*N
  for n in range(N):
    S[n] = S0*np.exp((r-sigma**2/2)*T+sigma*np.sqrt(T)*Z[n])
    payoff[n] = np.exp(-r*T)*max(K-S[n],0)
  MC_est = np.mean(payoff)
  MC_sd = statistics.stdev(payoff)
  h = MC_sd * st.t.ppf(0.975, N-1) / np.sqrt(N)
  return(MC_est,MC_est-h,MC_est+h)

# European Call
def MC_BSCall(S0,K,T,sigma,r,N):
  Z = np.random.normal(0,1,N)
  payoff = []
  for n in range(N):
    S_T = S0*np.exp((r-sigma**2/2)*T+sigma*np.sqrt(T)*Z[n])
    payoff += [np.exp(-r*T)*max(S_T-K,0)]
  MC_est = np.mean(payoff)
  MC_sd = statistics.stdev(payoff)
  h = MC_sd * st.t.ppf(0.975, N-1) / np.sqrt(N)
  return(MC_est,MC_est-h,MC_est+h)

In [ ]:
S0 = 3977.19; K = 3977.19; T = 1; sigma = 0.12; r = 0.05
# N = 100
MC_100 = MC_BSPut(S0,K,T,sigma,r,100)
print("The MC estimate with 100 realizations is $%.2f." % MC_100[0])
print("The 95%% CI with 100 realizations is (%.2f, %.2f)." % (MC_100[1],MC_100[2]))
# N = 1000
MC_1000 = MC_BSPut(S0,K,T,sigma,r,1000)
print("The MC estimate with 1000 realizations is $%.2f." % MC_1000[0])
print("The 95%% CI with 1000 realizations is (%.2f, %.2f)." % (MC_1000[1],MC_1000[2]))
# N = 10000
MC_10000 = MC_BSPut(S0,K,T,sigma,r,10000)
print("The MC estimate with 10000 realizations is $%.2f." % MC_10000[0])
print("The 95%% CI with 10000 realizations is (%.2f, %.2f)." % (MC_10000[1],MC_10000[2]))

The MC estimate with 100 realizations is $94.89.
The 95% CI with 100 realizations is (62.76, 127.02).
The MC estimate with 1000 realizations is $101.47.
The 95% CI with 1000 realizations is (90.05, 112.89).
The MC estimate with 10000 realizations is $103.49.
The 95% CI with 10000 realizations is (99.76, 107.22).


In [ ]:
def BS_PUT(S0, K, T, sigma, r):
  N = st.norm.cdf
  d1 = (np.log(S0/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
  d2 = d1 - sigma * np.sqrt(T)
  return K*np.exp(-r*T)*N(-d2) - S0*N(-d1)

def BS_CALL(S0, K, T, sigma, r):
  N = st.norm.cdf
  d1 = (np.log(S0/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
  d2 = d1 - sigma * np.sqrt(T)
  #return K*np.exp(-r*T)*N(-d2) - S0*N(-d1)
  return S0*N(d1) - K*np.exp(-r*T)*N(d2)

BS_PUT(S0,K,T,sigma,r)

104.5237876313463

In [ ]:
trueP = BS_PUT(S0,K,T,sigma,r)
print("The actual Black-Scholes price is $%.2f." % trueP)

The actual Black-Scholes price is $104.52.


In [ ]:
count = 0
for i in range(500):
  CI_low, CI_upp = MC_BSPut(S0,K,T,sigma,r,1000)[1:]
  if trueP > CI_low and trueP < CI_upp:
    count += 1
print("The BS Price falls into the confidence intervals for %d times." % count)

The BS Price falls into the confidence intervals for 470 times.


In [ ]:
# part d
def MC_BSPut_sd(S0,K,T,sigma,r,N):
  Z = np.random.normal(0,1,N)
  S = payoff = [0]*N
  for n in range(N):
    S[n] = S0*np.exp((r-sigma**2/2)*T+sigma*np.sqrt(T)*Z[n])
    payoff[n] = np.exp(-r*T)*max(K-S[n],0)
  MC_est = np.mean(payoff)
  MC_sd = statistics.stdev(payoff)
  h = MC_sd * st.t.ppf(0.975, N-1) / np.sqrt(N)
  return MC_sd

est_sd = np.mean([MC_BSPut_sd(S0,K,T,sigma,r,1000) for i in range(500)])

In [ ]:
# want h to be less than 0.02, h = MC_sd * 1.96 / np.sqrt(N), N>(est_sd*1.96/0.02)**2
N = int((est_sd*1.96/0.02)**2+1)
N

350660219

# Q3. Monte Carlo Option Pricing (II): Asian Option
Consider the same setup as in Q2, i.e., $S_0=K=3977.19,\ \sigma=16\%,\ r=5\%,\ T=1,$ and trading days $d=256.$ An **Asian Average Rate Call** Option is an option that, at expiry $T,$ pays
$$
(A-K)^+ \text{ where } A=\frac{1}{256}\sum_{i=1}^{256}S_{t_i},
$$
and $S_{t_i}$ is the stock price on day $i\in \{1,2,\cdots,256\}.$ Determine a Monte-Carlo estimate of the option price.

In [ ]:
S0 = 3977.19; K = 3977.19; T = 1; sigma = 0.12; r = 0.05; d = 252

# Generates N sample paths of the stock price
# returns a N x (d+1) matrix
def sample_path(S0, K, T, sigma, r, N, d):
  St_paths = np.ndarray(shape=(N,d+1)) # Initialize the stock paths
  for n in range(N): # For each simulation (stock path)
    Z = np.random.normal(0,1,d) # generate d standard normal random variables
    St_paths[n,0] = S0 # stock price at time 0
    for i in range(1,d+1): # recursion
      St_paths[n,i] = St_paths[n,i-1] * np.exp((r-sigma**2/2)*(T/d)+sigma*np.sqrt(T/d)*Z[i-1])
  return(St_paths[:,1:])

stock_paths = sample_path(S0,K,T,sigma,r,10000,d) #10000 sample paths of the stock price

In [ ]:
# Calculate realizations A_j and payoffs C_j
A = [np.average(x) for x in stock_paths]
C = [np.exp(-r*T)*max(x-K,0) for x in A]
print("An MC estimate of the option price is $%.2f." % np.average(C))

An MC estimate of the option price is $161.42.


In [ ]:
# Alternatively
S0 = 3977.19; K = 3977.19; T = 1; sigma = 0.12; r = 0.05; d = 256; N = 10000
C = [0]*N # C_j's
for n in range(N):
  St_path = [S0] # stock path for each simulation
  for i in range(1,d+1):
    Z = np.random.normal(0,1)
    St_path += [St_path[i-1]*np.exp((r-0.5*sigma**2)*(T/d)+sigma*np.sqrt(T/d)*Z)]
  A = np.average(St_path) # A_j
  C[n] = np.exp(-r*T) * max(A-K,0) # C_j

np.average(C)

158.59500695055567